In [1]:
### we start with the liar dataset

import numpy as np
import pandas as pd

In [2]:
#get the dataset
df_train = pd.read_csv('../../../datasets/liar_tweaked/trainvectordata.csv')
df_test = pd.read_csv('../../../datasets/liar_tweaked/testvectordata.csv')
df_valid = pd.read_csv('../../../datasets/liar_tweaked/validvectordata.csv')

In [3]:
X_train=df_train['statement']
X_test=df_test['statement']
Y_train=df_train['label']
Y_test=df_test['label']
X_valid=df_valid['statement']
Y_valid=df_valid['label']

In [4]:
X_test=X_test.append(X_valid, ignore_index=True)

In [5]:
Y_test=Y_test.append(Y_valid, ignore_index=True)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#make necessary imports
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression



Using TensorFlow backend.


In [7]:
#tfidf
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(X_train) 
tfidf_test=tfidf_vectorizer.transform(X_test)  

In [8]:
np.array(tfidf_test.todense().tolist()).shape

(2551, 11915)

In [9]:
tfidf_train = np.array(tfidf_train.todense().tolist())
tfidf_test = np.array(tfidf_test.todense().tolist())

In [10]:
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [11]:
#building the classifier
def build_classifier():
    clf=Sequential()
    clf.add(Dense(output_dim=500,init='uniform',activation='relu',input_dim=11915))
    clf.add(Dense(output_dim=100,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=50,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=20,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=10,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=5,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

In [ ]:
#make necessary imports
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

#build ANN, use k fold cross validation
clf=KerasClassifier(build_fn=build_classifier, batch_size=10, nb_epoch=100)
accuracies=cross_val_score(estimator=clf, X=tfidf_train,y=Y_train,cv=10,n_jobs=-1)

In [ ]:
accuracies

In [ ]:
#fit on training data and check accuracies on both test and valid data
clf.fit(tfidf_train,Y_train, batch_size=10, nb_epoch=10)
y_test_pred = clf.predict(tfidf_test)
print('algorithm - test dataset accuracy - valid dataset accuracy')
print('ANNTFIDF - ' ,round(accuracy_score(Y_test, y_test_pred),4), ' - ', end='')

In [ ]:
history = clf.fit(tfidf_train, Y_train, epochs=10,
                    validation_data=(tfidf_test, Y_test))

In [20]:
df = pd.read_csv('../../../datasets/kaggle/train.csv')

In [21]:
df['content'] = df['title'] + ' ' + df['text']
df=df[df['content']==df['content']]
df=df[df['label']==df['label']]

In [22]:
df.to_csv('../../../datasets/kaggle/final.csv')

In [23]:
df=pd.read_csv('../../../datasets/kaggle/final.csv')

In [24]:
X=df['content']
Y=df['label']

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [26]:
X_test=X_test.reset_index()
Y_test=Y_test.reset_index()
X_train=X_train.reset_index()
Y_train=Y_train.reset_index()

In [27]:
Y_test=Y_test['label']
X_test=X_test['content']
X_train=X_train['content']
Y_train=Y_train['label']

In [28]:
#tfidf
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(X_train) 
tfidf_test=tfidf_vectorizer.transform(X_test)

In [29]:
tfidf_test = tfidf_test.toarray()

In [30]:
tfidf_train=tfidf_train.toarray()

In [31]:
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [37]:
tfidf_test.shape

(5051, 157102)

In [38]:
#building the classifier
def build_classifier():
    clf=Sequential()
    clf.add(Dense(output_dim=500,init='uniform',activation='relu',input_dim=157102))
    clf.add(Dense(output_dim=100,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=50,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=20,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=10,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=5,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

In [39]:
clf=KerasClassifier(build_fn=build_classifier, batch_size=10, nb_epoch=100)

In [ ]:
#fit on training data and check accuracies on both test and valid data
history = clf.fit(tfidf_train, Y_train, epochs=10,
                    validation_data=(tfidf_test, Y_test))